In [1]:
using BenchmarkTools, LinearAlgebra

In [2]:
function Ls(L::Matrix{T}, b::Vector{T}) where T<:Number
    n = size(L)[1]
    x = zeros(T, n)
    x[1] = b[1]/L[1, 1]
    for i in 2:1:n
        x[i] = b[i]
        for j in 1:1:(i-1)
            x[i] -= L[i, j]*x[j]
        end
        x[i] = x[i]/L[i, i]
    end
    return x
end

function Us(U::Matrix{T}, b::Vector{T})::Vector{T} where T<:Number
    n = size(U)[1]
    x = zeros(T, n)
    x[n] = b[n]/U[n, n]
    for i in (n-1):-1:1
        x[i] = b[i]
        for j in (i+1):1:n
            x[i] -= U[i, j] * x[j]
        end
        x[i] = x[i]/U[i, i]
    end
    return x
end


Us (generic function with 1 method)

In [3]:
A = [1. 2. 3. 5.; 0. 2. 1. -2.; 0. 0. -3. 3.; 0. 0. 0. 7.]
b = [3.1; 5.3; -2.2; 6.0]
x = Us(A, b) 

4-element Vector{Float64}:
 -11.38095238095238
   2.711904761904762
   1.5904761904761904
   0.8571428571428571

In [4]:
A*x

4-element Vector{Float64}:
  3.1000000000000005
  5.3
 -2.2
  6.0

In [5]:
A1= [0.531656 1.18632e7 1.0599e7; 0.0 0.25809 1.40481e7 ;0.0 0.0 0.846038]
b1 = [3.1; 5.3; -2.2]
x1 = Us(A1, b1)

3-element Vector{Float64}:
 -3.1582784131957885e15
  1.41540030987187e8
 -2.600356012377695

In [6]:
A1*x1


3-element Vector{Float64}:
  3.1248088106513023
  5.299999997019768
 -2.2

In [7]:
function GE1(M::Matrix{T})::Array{T, 2} where T<:Number
    A = copy(M)
    m, n = size(A)[1:2]
    M = min(m, n)
    for i in 1:1:M, j in (i+1):1:m
        A[j, :] = A[j, :] .- (A[i, :] .* (A[j , i]/A[i, i]))
    end
    return A
end

GE1 (generic function with 1 method)

In [8]:
A1 = Float64.([1 3 2; 3 -4 2; 3 3 1])
GE1(A1)

3×3 Matrix{Float64}:
 1.0    3.0   2.0
 0.0  -13.0  -4.0
 0.0    0.0  -3.15385

In [9]:
A2 = Float64.([1 2 3 ; 2 3 4 ; 3 4 5; 1 2 2])
GE1(A2)

4×3 Matrix{Float64}:
   1.0    2.0    3.0
   0.0   -1.0   -2.0
   0.0    0.0    0.0
 NaN    NaN    NaN

In [11]:
function find_nonzero_rows_after(M::Matrix{T}, i::Integer) where T<:Number
    h, w = size(M)
    k = 0
    for j in i+1:1:h
        if M[j, i] ≠ 0.0
            k = j
            break
        end
    end
    return k
end

function GE2(M::Array{T, 2})::Matrix{T, 2} where T<:Number
    A = copy(M)
    m, n = size(A)[1:2]
    M = min(m, n)
    for i in 1:1:M, j in (i+1):1:m
        if A[i, i] == 0.0
            k = find_nonzero_rows_after(A, i)
            if k ≠ nothing
                A[i, :], A[k, :] = A[k, :], A[i, :]
            else
                continue
            end
        end
        A[j, :] = A[j, :] .- (A[i, :] .* (A[j , i]/A[i, i]))
        A[j, i] = 0.0
    end
    return A
end

GE2 (generic function with 1 method)